In [155]:
import pandas
import numpy as np
from sklearn.metrics import roc_auc_score

In [9]:
data = pandas.read_csv("data-logistic.csv", header=None)

In [32]:
target = np.array(data[0])
features = []
for i in range(len(target)):
    test = []
    test.append(data[1][i])
    test.append(data[2][i])
    features.append(test)
features = np.array(features)
print(features.T.shape)

(2, 205)


In [149]:
def changeW_L2(w, f, t, k, C):
    
    w0 = []
    temp1 = 0
    temp2 = 0
    for i in range(len(t)):
        temp1 += t[i]*f.T[0][i]*(1 - 1/(1+np.exp(-t[i]*(w[0]*f[i][0] + w[1]*f[i][1])))) - k*C*w[0]
        temp2 += t[i]*f.T[1][i]*(1 - 1/(1+np.exp(-t[i]*(w[0]*f[i][0] + w[1]*f[i][1])))) - k*C*w[1]
    r = (w[0] + temp1*k/len(t))
    w0.append(w[0] + temp1*k/len(t))
    w0.append(w[1] + temp2*k/len(t))
    w0 = np.array(w0)  
    return w0

def changeW(w, f, t, k):
    
    w0 = []
    temp1 = 0
    temp2 = 0
    for i in range(len(t)):
        temp1 += t[i]*f.T[0][i]*(1 - 1/(1+np.exp(-t[i]*(w[0]*f[i][0] + w[1]*f[i][1]))))
        temp2 += t[i]*f.T[1][i]*(1 - 1/(1+np.exp(-t[i]*(w[0]*f[i][0] + w[1]*f[i][1]))))
    r = (w[0] + temp1*k/len(t))
    w0.append(w[0] + temp1*k/len(t))
    w0.append(w[1] + temp2*k/len(t))
    w0 = np.array(w0)              
    return w0

def error(w, f, t):
    
    M = (-1)*np.dot(f, w.T)*t
    #print(np.exp(M))
    err = np.sum(np.log(1+np.exp(M)))/len(t)
    return err

def error_L2(w,f,t,c):
    M = (-1)*np.dot(f, w.T)*t
    err = np.sum(np.log(1+np.exp(M)))/len(t) + C/2*np.linalg.norm(w)**2
    return err

In [219]:
k = 10
C = 10
w0 = np.array([0, 0])
err = error(w0, features, target)
#print(err)
w = changeW(w0, features, target,k)
err_2 = error(w, features, target)

for j in range(10000):
    if(np.sqrt(np.sum(np.abs(w0-w)**2)) >1e-5):
        w0 = w
        w = changeW(w,features,target,k)
        #err_2 = error(w, features, target)
    else:
        break
err = error(w, features, target)
print(err, j)

0.6873294386565795 9999


In [216]:
w0l = w0
wl = changeW_L2(w0, features, target, k, C)
for j in range(10000):
    if(np.sqrt(np.sum(np.abs(w0l-wl)**2)) >1e-5):
        w0l = wl
        wl = changeW_L2(wl,features,target,k, C)
        #print(wl)
    else:
        break
err = error_L2(wl, features, target, C)
print(errl, j)

0.7619513402260516 322


In [217]:
print(w, wl)
M = np.dot(features, w)
a = 1/(1+np.exp(-M))
M_l2 = np.dot(features, wl)
a_l2 = 1/(1+np.exp(-M_l2))

[0.28808965 0.09172625] [nan nan]


In [218]:
print(roc_auc_score(target, a))
print(roc_auc_score(target, a_l2))

0.9267619047619048


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').